In [1]:
# --- Podstawowe biblioteki ---
import pandas as pd
import numpy as np
from datetime import datetime
import os

# --- Biblioteki do Machine Learningu i śledzenia eksperymentów ---
from pycaret.regression import setup, compare_models, tune_model, finalize_model, save_model, predict_model, pull

In [2]:
# --- Wczytywanie Danych (Logika dostosowana do istniejącego Predict_State) ---

# 1. Wczytanie głównego zbioru danych (który już zawiera Predict_State)
try:
    print("Wczytuję kompletny plik danych: Data_state_LSTM_predicted_full.csv...")
    df_original = pd.read_csv('Data_state_LSTM_predicted_full.csv', sep=';', low_memory=False)
    print(f"Wczytano pomyślnie. Kształt danych: {df_original.shape}")
except Exception as e:
    print(f"BŁĄD: Nie można wczytać głównego pliku danych. {e}")
    df_original = pd.DataFrame()

# 2. Wczytanie pliku z predykcjami lokalizacji w celu dodania kolumny 'Predict_Loc'
try:
    print("\nWczytuję plik z predykcjami lokalizacji: Location_Polska.csv...")
    # Używamy średnika jako separatora, zgodnie z tym jak plik jest generowany
    df_locations = pd.read_csv('Location_Polska.csv', sep=';', engine='python')
    # Wybieramy tylko kolumny potrzebne do połączenia
    df_locations = df_locations[['Title', 'Description', 'Predict_Loc']]
    print(f"Predykcje lokalizacji wczytane pomyślnie. Kształt danych: {df_locations.shape}")
except Exception as e:
    print(f"BŁĄD: Nie można wczytać pliku Location_Polska.csv. {e}")
    df_locations = pd.DataFrame()

# --- Łączenie danych w celu dodania 'Predict_Loc' ---
if not df_original.empty and not df_locations.empty:
    print("\nPrzygotowuję dane do połączenia...")
    
    # Tworzymy unikalny klucz do łączenia oparty na tytule i opisie
    df_original['merge_key'] = df_original['Title'].fillna('') + df_original['Description'].fillna('')
    df_locations['merge_key'] = df_locations['Title'].fillna('') + df_locations['Description'].fillna('')
    
    # Usuwamy duplikaty klucza w pliku lokalizacji, aby uniknąć powielenia wierszy po merge
    df_locations.drop_duplicates(subset=['merge_key'], keep='first', inplace=True)

    print("Łączę predykcję 'Predict_Loc' z głównym zbiorem danych...")
    # Używamy 'left merge', aby dołączyć Predict_Loc do df_original
    df_original = pd.merge(
        df_original, 
        df_locations[['merge_key', 'Predict_Loc']], 
        on='merge_key', 
        how='left'
    )
    
    # Sprzątanie i uzupełnianie braków
    df_original.drop(columns=['merge_key'], inplace=True, errors='ignore')
    df_original['Predict_Loc'].fillna('Brak predykcji', inplace=True)
    # Na wszelki wypadek uzupełniamy też braki w istniejącej kolumnie Predict_State
    df_original['Predict_State'].fillna('Brak Danych', inplace=True)
    
    print("\nPołączenie zakończone.")
    print(f"Nowy kształt danych: {df_original.shape}")
    print(f"Liczba wierszy z przypisaną lokalizacją (Predict_Loc): {(df_original['Predict_Loc'] != 'Brak predykcji').sum()}")
    print(f"Liczba wierszy z przypisanym województwem (Predict_State): {(df_original['Predict_State'] != 'Brak Danych').sum()}")

else:
    # Zabezpieczenie, gdyby któryś z plików nie został wczytany
    print("\nBŁĄD: Nie udało się wczytać wszystkich plików. Tworzę pustą kolumnę 'Predict_Loc'.")
    if 'Predict_Loc' not in df_original.columns:
        df_original['Predict_Loc'] = np.nan

Wczytuję kompletny plik danych: Data_state_LSTM_predicted_full.csv...
Wczytano pomyślnie. Kształt danych: (760765, 55)

Wczytuję plik z predykcjami lokalizacji: Location_Polska.csv...
Predykcje lokalizacji wczytane pomyślnie. Kształt danych: (783716, 3)

Przygotowuję dane do połączenia...
Łączę predykcję 'Predict_Loc' z głównym zbiorem danych...

Połączenie zakończone.
Nowy kształt danych: (760765, 56)
Liczba wierszy z przypisaną lokalizacją (Predict_Loc): 753783
Liczba wierszy z przypisanym województwem (Predict_State): 760765


In [3]:
# KOREKTA: Funkcja prepare_data_for_modeling została poprawiona
# Usunięto błędną logikę łączenia stanu budynku z lokalizacją.
def prepare_data_for_modeling(input_df):
    """
    Kompleksowo przygotowuje dane: inflacja, typy, wiek budynku
    oraz czyści kolumny Predict_State (stan budynku) i Predict_Loc (lokalizacja).
    """
    df = input_df.copy()
    
    # 1. Inżynieria Cech (Inflacja) - bez zmian
    if 'AdjustedPrice' not in df.columns:
        print("Tworzenie kolumny 'AdjustedPrice'...")
        def adjust_price(row):
            price = pd.to_numeric(row['Price'], errors='coerce')
            if pd.isna(price): return np.nan
            date_str = row['NewestDate'] if 'NewestDate' in row and pd.notna(row['NewestDate']) else (row['DateAdded'] if 'DateAdded' in row and pd.notna(row['DateAdded']) else None)
            if date_str is None or date_str == 'NULL': return price
            try:
                offer_date = pd.to_datetime(date_str, errors='coerce')
                if pd.isna(offer_date): return price
                years_diff = (datetime.now() - offer_date).days / 365.25
                return round(price * (1.05**years_diff), 0) if years_diff > 0 else price
            except: return price
        df['AdjustedPrice'] = df.apply(adjust_price, axis=1)

    # 2. Konwersja typów i tworzenie 'BuildingAge' - bez zmian
    for col in ['Area', 'NumberOfRooms', 'Floor', 'Floors']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    if 'BuiltYear' in df.columns:
        print("Konwertowanie 'BuiltYear' i tworzenie 'BuildingAge'...")
        median_year = pd.to_numeric(df['BuiltYear'], errors='coerce').median()
        df['BuiltYear_numeric'] = pd.to_numeric(df['BuiltYear'], errors='coerce').fillna(median_year)
        df['BuildingAge'] = datetime.now().year - df['BuiltYear_numeric']
        df['BuiltYear'] = df['BuiltYear_numeric'].astype(int).astype(str)
        df['BuiltYear'] = pd.to_datetime(df['BuiltYear'], format='%Y', errors='coerce')
        df.drop(columns=['BuiltYear_numeric'], inplace=True)

    # KOREKTA: Poprawiona obsługa Predict_State i Predict_Loc
    # Teraz są traktowane jako dwie ODDZIELNE cechy kategoryczne.
    print("Czyszczenie kolumn 'Predict_State' (stan budynku) i 'Predict_Loc' (lokalizacja)...")
    if 'Predict_State' in df.columns:
        df['Predict_State'].fillna('Brak Danych', inplace=True)
    if 'Predict_Loc' in df.columns:
        df['Predict_Loc'].fillna('Brak Danych', inplace=True)
    
    print("Przetwarzanie danych zakończone.")
    return df

# --- Główny przepływ -- (bez zmian w tej części)
if not df_original.empty:
    df_processed = prepare_data_for_modeling(df_original)
    
    data_for_training = df_processed.dropna(subset=['AdjustedPrice', 'Area']).copy()
    print(f"\nLiczba wierszy (przed czyszczeniem outlierów): {len(data_for_training)}")

    for col in ['AdjustedPrice', 'Area']:
        Q1 = data_for_training[col].quantile(0.25)
        Q3 = data_for_training[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        data_for_training = data_for_training[(data_for_training[col] >= lower_bound) & (data_for_training[col] <= upper_bound)]
    
    print(f"Liczba wierszy do treningu (po czyszczeniu): {len(data_for_training)}")
else:
    print("BŁĄD: Zbiór 'df_original' jest pusty. Przerwanie przetwarzania.")
    data_for_training = pd.DataFrame()

Tworzenie kolumny 'AdjustedPrice'...
Konwertowanie 'BuiltYear' i tworzenie 'BuildingAge'...
Czyszczenie kolumn 'Predict_State' (stan budynku) i 'Predict_Loc' (lokalizacja)...
Przetwarzanie danych zakończone.

Liczba wierszy (przed czyszczeniem outlierów): 760765
Liczba wierszy do treningu (po czyszczeniu): 692613


In [4]:
# NLP Feature Engineering (bez zmian)
from sklearn.feature_extraction.text import TfidfVectorizer

if not data_for_training.empty:
    print("Rozpoczynam tworzenie cech z opisów (TF-IDF)...")
    
    data_for_training['Description_clean'] = data_for_training['Description'].fillna('').str.lower()
    
    vectorizer = TfidfVectorizer(max_features=100, stop_words=['i', 'w', 'na', 'z', 'do', 'jest', 'oraz', 'mieszkanie', 'mieszkania', 'zl'])
    
    tfidf_features = vectorizer.fit_transform(data_for_training['Description_clean'])
    
    df_tfidf = pd.DataFrame(tfidf_features.toarray(), columns=[f"tfidf_{name}" for name in vectorizer.get_feature_names_out()])
    df_tfidf.index = data_for_training.index
    
    data_for_training = pd.concat([data_for_training, df_tfidf], axis=1)
    
    print(f"Dodano {df_tfidf.shape[1]} nowych cech z opisów.")
    print(f"Nowy kształt danych do treningu: {data_for_training.shape}")
else:
    print("Zbiór 'data_for_training' jest pusty, pomijam NLP.")

Rozpoczynam tworzenie cech z opisów (TF-IDF)...
Dodano 100 nowych cech z opisów.
Nowy kształt danych do treningu: (692613, 159)


In [5]:
# NOWA KOMÓRKA - ZAPIS WEKTORYZATORA
import joblib

# Zapisujemy obiekt wektoryzatora do pliku, aby móc go użyć w skrypcie predykcyjnym
vectorizer_filename = 'tfidf_vectorizer.pkl'
joblib.dump(vectorizer, vectorizer_filename)

print(f"Wektoryzator TF-IDF został pomyślnie zapisany do pliku: {vectorizer_filename}")

Wektoryzator TF-IDF został pomyślnie zapisany do pliku: tfidf_vectorizer.pkl


In [6]:
if not data_for_training.empty:
    print("--- Konfiguracja eksperymentu PyCaret ---\n")
    
    numeric_features = ['Area', 'NumberOfRooms', 'Floor', 'Floors', 'BuildingAge']
    date_features = ['BuiltYear']
    
    # KOREKTA: Zaktualizowana lista cech kategorycznych
    # Usunięto 'Unified_Location', dodano 'Predict_Loc'. 'Predict_State' zostaje, ale teraz ma poprawne znaczenie.
    categorical_features = ['BuildingType', 'TypeOfMarket', 'Type', 'OfferFrom', 'OwnerType', 'Predict_State', 'Predict_Loc']

    print(f"Finalna lista cech kategorycznych do użycia w modelu: {categorical_features}\n")
    
    # KOREKTA: Zaktualizowana lista ignorowanych cech
    # Usunięto 'Predict_Loc' z ignorowanych. Dodano 'Unified_Location' na wszelki wypadek.
    ignore_features = [
        'SaleId', 'OriginalId', 'PortalId', 'Title', 'Description', 'Description_clean',
        'BuildingCondition', 'Price',
        'DateAddedToDatabase', 'DateAdded', 'NewestDate', 'DateLastModification', 'DateLastRaises',
        'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice', 'PricePerSquareMeter',
        'AvailableFrom', 'Link', 'Phone', 'MainImage', 'OtherImages', 'NumberOfDuplicates',
        'NumberOfRaises', 'NumberOfModifications', 'IsDuplicatePriceLower', 'IsDuplicatePrivateOwner',
        'Score', 'ScorePrecision', 'CommunityScore', 'NumberOfCommunityComments', 'NumberOfCommunityOpinions',
        'Archive', 'Location', 'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber',
        'RegionNumber', 'SubRegionNumber', 'StreetNumber', 'EncryptedId', 'PredictedRenovation',
        'LocationPath', 'Unified_Location' # Dodane na wypadek, gdyby kolumna została stworzona przez pomyłkę
    ]

    numeric_features_to_use = [c for c in numeric_features if c in data_for_training.columns]
    categorical_features_to_use = [c for c in categorical_features if c in data_for_training.columns]
    date_features_to_use = [c for c in date_features if c in data_for_training.columns]
    ignore_features_to_use = [c for c in ignore_features if c in data_for_training.columns]

    print(f"Cechy numeryczne: {numeric_features_to_use}")
    print(f"Cechy kategoryczne: {categorical_features_to_use}")
    print(f"Cechy daty: {date_features_to_use}")
    print(f"Liczba ignorowanych cech: {len(ignore_features_to_use)}")

    # Inicjalizacja środowiska PyCaret
    reg_exp = setup(
        data=data_for_training,
        target='AdjustedPrice',
        session_id=123,
        log_experiment=False, 
        numeric_features=numeric_features_to_use,
        categorical_features=categorical_features_to_use,
        date_features=date_features_to_use,
        ignore_features=ignore_features_to_use,
        normalize=True,
        normalize_method='zscore'
    )
else:
    print("BŁĄD: Zbiór 'data_for_training' jest pusty. Nie można uruchomić setup().")

--- Konfiguracja eksperymentu PyCaret ---

Finalna lista cech kategorycznych do użycia w modelu: ['BuildingType', 'TypeOfMarket', 'Type', 'OfferFrom', 'OwnerType', 'Predict_State', 'Predict_Loc']

Cechy numeryczne: ['Area', 'NumberOfRooms', 'Floor', 'Floors', 'BuildingAge']
Cechy kategoryczne: ['BuildingType', 'TypeOfMarket', 'Type', 'OfferFrom', 'OwnerType', 'Predict_State', 'Predict_Loc']
Cechy daty: ['BuiltYear']
Liczba ignorowanych cech: 44


,Description,Value
0,Session id,123
1,Target,AdjustedPrice
2,Target type,Regression
3,Original data shape,"(692613, 159)"
4,Transformed data shape,"(692613, 123)"
5,Transformed train set shape,"(484829, 123)"
6,Transformed test set shape,"(207784, 123)"
7,Ignore features,44
8,Numeric features,5
9,Date features,1


In [7]:
# --- Bezpieczne Porównanie Modeli ---

if 'reg_exp' in locals() and reg_exp is not None:
    print("Rozpoczynam porównywanie wszystkich dostępnych modeli. To może potrwać...")
    best_models_list = compare_models(sort='R2', n_select=3)

    if best_models_list:
        best_model = best_models_list[0]
        print("\n--- Najlepsze znalezione modele ---")
        print(best_models_list)
        print("\n--- Wybrany najlepszy model do dalszej pracy ---")
        print(best_model)
    else:
        print("\nBŁĄD: compare_models nie zwróciło żadnych modeli.")
        best_model = None
else:
    print("BŁĄD KRYTYCZNY: Eksperyment PyCaret (setup) nie został uruchomiony.")
    best_model = None

Rozpoczynam porównywanie wszystkich dostępnych modeli. To może potrwać...


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,67436.8560,11409266709.9722,106812.4135,0.8095,0.4150,41.2207,122.6710
catboost,CatBoost Regressor,80069.1202,13413619078.9852,115815.0682,0.7760,0.4216,43.8358,13.6770
xgboost,Extreme Gradient Boosting,82320.0219,14011010867.2000,118366.1039,0.7661,0.4250,43.4062,5.2100
lightgbm,Light Gradient Boosting Machine,85440.8094,14966118805.9133,122333.8634,0.7501,0.4308,43.8847,3.7100
dt,Decision Tree Regressor,91396.0512,22582510207.0220,150271.8974,0.6229,0.5725,39.6321,7.2880
dummy,Dummy Regressor,193386.4844,59892005683.2000,244727.1703,-0.0000,0.6179,53.9609,1.4640



--- Najlepsze znalezione modele ---
[RandomForestRegressor(n_jobs=-1, random_state=123), <catboost.core.CatBoostRegressor object at 0x000001B3256A0D90>, XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=-1, num_parallel_tree=None, ...)]

--- Wybrany najlepszy model do dalszej pracy ---
RandomForestRegressor(n_jobs=-1, random_st

In [8]:
# --- KROK: TUNE_MODEL ---

if 'best_model' in locals() and best_model is not None:
    print(f"Rozpoczynam tuning najlepszego modelu: {type(best_model).__name__}")
    tuned_best_model = tune_model(best_model, n_iter=20, optimize = 'R2')
    
    print("\n--- Wyniki po tuningu ---")
    tuned_results = pull()
    display(tuned_results)
    
    best_model = tuned_best_model 
else:
    print("BŁĄD: Zmienna 'best_model' nie została znaleziona lub jest pusta.")

Rozpoczynam tuning najlepszego modelu: RandomForestRegressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,92418.4016,17411985422.8771,131954.4824,0.7098,0.4588,39.2536
1,92280.0679,17258911982.2355,131373.1783,0.7155,0.4466,58.5447
2,92015.0020,17075660043.7477,130673.8690,0.7125,0.4316,15.7294
3,92460.2787,17304749101.5672,131547.5165,0.7140,0.4445,42.4625
4,91995.6090,16985637809.2064,130328.9600,0.7151,0.4403,36.3267
5,92670.7263,17309481682.9497,131565.5034,0.7108,0.4550,30.2203
6,92692.3622,17327793650.6735,131635.0776,0.7094,0.4679,77.2512
7,92425.3177,17266837998.5813,131403.3409,0.7108,0.4481,61.8894
8,91623.7182,16937117219.4515,130142.6802,0.7161,0.4486,53.4873


Fitting 10 folds for each of 20 candidates, totalling 200 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).

--- Wyniki po tuningu ---


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,92418.4016,1.741199e+10,131954.4824,0.7098,0.4588,39.2536
1,92280.0679,1.725891e+10,131373.1783,0.7155,0.4466,58.5447
2,92015.0020,1.707566e+10,130673.8690,0.7125,0.4316,15.7294
3,92460.2787,1.730475e+10,131547.5165,0.7140,0.4445,42.4625
4,91995.6090,1.698564e+10,130328.9600,0.7151,0.4403,36.3267
5,92670.7263,1.730948e+10,131565.5034,0.7108,0.4550,30.2203
6,92692.3622,1.732779e+10,131635.0776,0.7094,0.4679,77.2512
7,92425.3177,1.726684e+10,131403.3409,0.7108,0.4481,61.8894
8,91623.7182,1.693712e+10,130142.6802,0.7161,0.4486,53.4873


In [9]:
# --- Finalizacja, Zapis i Predykcja (z drobną modyfikacją wywołania funkcji) ---

if 'best_model' in locals() and best_model is not None:
    print("Finalizowanie najlepszego modelu...")
    final_model = finalize_model(best_model)
    
    print("\nPrzygotowuję oryginalny zbiór danych do predykcji...")
    # ZMIANA: Wywołanie funkcji prepare_data_for_modeling bez słownika
    data_for_prediction = prepare_data_for_modeling(df_original)
    
    print("Stosowanie transformacji TF-IDF na danych do predykcji...")
    data_for_prediction['Description_clean'] = data_for_prediction['Description'].fillna('').str.lower()
    tfidf_features_pred = vectorizer.transform(data_for_prediction['Description_clean'])
    df_tfidf_pred = pd.DataFrame(tfidf_features_pred.toarray(), columns=[f"tfidf_{name}" for name in vectorizer.get_feature_names_out()])
    df_tfidf_pred.index = data_for_prediction.index
    data_for_prediction = pd.concat([data_for_prediction, df_tfidf_pred], axis=1)

    if 'AdjustedPrice' in data_for_prediction.columns:
        data_for_prediction_clean = data_for_prediction.drop(columns=['AdjustedPrice'])
    else:
        data_for_prediction_clean = data_for_prediction
    
    all_data_predictions = predict_model(final_model, data=data_for_prediction_clean)
    print("\nPredykcja zakończona.")

    all_data_predictions.rename(columns={'prediction_label': 'PredictedPrice'}, inplace=True)
    
    df_final_output = df_original.copy()
    df_final_output['AdjustedPrice'] = data_for_prediction['AdjustedPrice']
    df_final_output['PredictedPrice'] = all_data_predictions['PredictedPrice']
    
    output_filename = 'sale_2024_0_predict.csv'
    df_final_output.to_csv(output_filename, index=False, sep=';', decimal='.')

    print(f"\nWyniki zostały zapisane do pliku: {output_filename}")
    print("\nPrzykładowe dane z finalnego pliku:")
    display(df_final_output[['SaleId', 'Price', 'AdjustedPrice', 'PredictedPrice']].head(10))
else:
    print("BŁĄD: Zmienna 'best_model' nie została znaleziona lub jest pusta.")

Finalizowanie najlepszego modelu...

Przygotowuję oryginalny zbiór danych do predykcji...
Tworzenie kolumny 'AdjustedPrice'...
Konwertowanie 'BuiltYear' i tworzenie 'BuildingAge'...
Czyszczenie kolumn 'Predict_State' (stan budynku) i 'Predict_Loc' (lokalizacja)...
Przetwarzanie danych zakończone.
Stosowanie transformacji TF-IDF na danych do predykcji...



Predykcja zakończona.

Wyniki zostały zapisane do pliku: sale_2024_0_predict.csv

Przykładowe dane z finalnego pliku:


,SaleId,Price,AdjustedPrice,PredictedPrice
0,88,766500.0,827583.0,797824.7840
1,99,540000.0,580779.0,578582.7025
2,115,540000.0,580314.0,572607.6700
3,140,544000.0,567530.0,637582.5000
4,145,459000.0,491754.0,487052.3600
5,159,779000.0,835927.0,789841.8600
6,165,359000.0,386472.0,365464.1000
7,173,380000.0,407171.0,382519.3400
8,189,354000.0,379616.0,375157.1600
9,208,820000.0,881570.0,769721.2000
